# Requirements

In [40]:
import sys

# Problem setting

Python as a type `list` that works peerfectly well, and the implementation is well-optimized.  However, Python lists are ephemeral, not persistent, i.e., if you, e.g., append an element to a list, the original list no longer exists. In other words, the `append` method has side effects.

While this is fine (and likely expected) for an imperative programming language, this is not acceptable in functional programming where you expect functions to be side effect-free.

For persistent lists, some operations will have to copy part of the data structure, not the values stored in that data structure, but pats of the structure itself.  This will of course have a performance impact.  On the upside, this implies that persistent data structures can be used without hassle in parallel programs.

Data structures can be made persistent, i.e., they support multiple versions.  Here this is illustrated by the implementation of a persistent list.

# Data structure

A list will be represented using `tuple` and `None`.  The empty list is represented by `None`, a list with a single element the 2-tuple `('a', None)`, while a list with two elements is a 2-tuple that has the first element of the list as its first element, the second element of the list as its second, under the form of a tuple: `('a', ('b', None))`

More generally, a list is represented by a tuple, the first element is the head of the list, the second the tail, i.e., all other elements in a list, so for a list with four elements:
```
('a', ('b', ('c', ('d', None))))
```

Since a `tuple` in Python is immutable, persistence is already partly guaranteed since once created, a tuple can not be modified.

# Basic operations

You need a constructor for an empty list, as well as a test to check whether a list is empty.  In addition, you need three basic functions for persistence lists, and those will let you implement all other list operations:
* `empty()`: construct an empty list;
* `is_empty(xs)`: returns `True` if the list is empty, `False` otherwise;
* `cons(x, xs)`: construct a list by prepending the element `x` to the list `xs`;
* `head(xs)`: return the head of the list `xs`, i.e., its first element;
* `tail(xs)`: return the tail of the list, i.e., a list containing all elements, except the first.

Applying `head` or `tail` to an empty list should raise an exception, `ValueError` is appropriate.  

In [75]:
def empty():
    '''Create an empty list
    
    Returns
    -------
    None
        empty list
    '''
    return None

In [73]:
def is_empty(xs):
    '''Returns True if the list is empty, false otherwise
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to check for emptiness
        
    Returns
    -------
    bool
        True if the list is empty, false otherwise
    '''
    return xs is None

In [77]:
def cons(x, xs):
    '''Construct a list with x as head and xs as tail
    
    Parameters
    ----------
    x: Any
        value to be the first element in the new list
    xs: tuple[Any, tuple | None]
        list that is the tail of the new list
    
    Returns
    -------
    tuple[Any, tuple | None]
        new list
    '''
    return (x, xs)

In [78]:
def head(xs):
    '''Return the head of the string
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to get the first element of
        
    Returns
    -------
    Any
        value of the first element in the list
        
    Raises
    ------
    ValueError
        if the list is empty, i.e., xs == None
    '''
    if is_empty(xs):
        raise ValueError('empty list')
    return xs[0]

In [79]:
def tail(xs):
    '''Return the tail of the string
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to get the tail of as a list
        
    Returns
    -------
    tuple[Any, tuple | None] | None
        tail of the list, note that the tail of a single-element list is the empty list,
        so None
        
    Raises
    ------
    ValueError
        if the list is empty, i.e., xs == None
    '''

    if is_empty(xs):
        raise ValueError('empty list')
    return xs[1]

First verify that the empty list is empty.

In [110]:
is_empty(empty())

True

To test `cons`, `head` and `tail`, you can construct list of lengths up to some value.  For each, except the empty one, show the head and the tail.

In [111]:
lists = [empty()]
for i in range(4, 0, -1):
    lists.append(cons(i, lists[-1]))
for l in lists:
    if is_empty(l):
        print(f'{l} empty')
    else:
        print(f'{l}\n\thead = {head(l)}\n\ttail = {tail(l)}')

None empty
(4, None)
	head = 4
	tail = None
(3, (4, None))
	head = 3
	tail = (4, None)
(2, (3, (4, None)))
	head = 2
	tail = (3, (4, None))
(1, (2, (3, (4, None))))
	head = 1
	tail = (2, (3, (4, None)))


Check that the expected exceptions are raised when `head` and `tail` are applied to empty lists.

In [84]:
try:
    head(empty())
except ValueError as e:
    print(f'Expected exception raised: {e}')

Expected exception raised: empty list


In [85]:
try:
    tail(empty())
except ValueError as e:
    print(f'Expected exception raised: {e}')

Expected exception raised: empty list


Note that although the elements were "inserted" in the previous list, those lists were not modified, hence they are persistent, as required.

# None basic operations

Using `empty`, `is_empty`, `cons`, `head` and `tail`, we can define every list operation we like.  We define a few lists that we can use to test the implementations.

In [89]:
l1 = cons(1, cons(2, cons(3, cons(4, empty()))))

In [90]:
l2 = cons('a', cons('b', cons('c', empty())))

In [155]:
l3 = cons(3, cons(-1, cons(7, cons(0, cons(2, empty())))))

## Iterator over list elements

Let's first implement an iterator over the elements of the list since this makes some operations more convenient.

In [88]:
def elements(xs):
    '''Iterate over the elements of a list
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
    
    Returns
    -------
    Any | None
        elements of the list, None when all elements were returned
    '''
    if not is_empty(xs):
        yield xs[0]
        yield from elements(tail(xs))

Check by printing all elements of the list.

In [91]:
for element in elements(l1):
    print(element)

1
2
3
4


Check whether this works for empty lists too.

In [112]:
for element in elements(empty()):
    print('Oops!')

## Flatten

This is a function that can be used to simplify testing and visualization, it will convert a persistent list of $N$ elements into an $N$-tuple that has the same elements in the same order.

In [92]:
def flatten(xs):
    '''Flatten a list into a tuple
    
    Parameeters
    -----------
    xs: tuple[Any, tuple | None] | None
        list to flatten
        
    Returns
    -------
    tuple[Any] | None
        tuple representing the list or an empty list if the original list was empty
    '''
    if is_empty(xs):
        return xs
    return tuple(x for x in elements(xs))

In [93]:
flatten(l1)

(1, 2, 3, 4)

## Length

A function to determine the length of the list is easy to write.

In [95]:
def length(xs):
    '''Return the length of the list
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to determine the length of
        
    Returns
    -------
    int
        length of the list
    '''
    if is_empty(xs):
        return 0
    return 1 + length(tail(xs))

In [113]:
length(empty())

0

In [97]:
length(l1)

4

In [98]:
length(l2)

3

## Element by index

We implement a function that returns an element from a list at a given index.

In [94]:
def element_at(xs, i):
    '''Get the element at the given index position
    
    Parameters
    ----------
    xs: tuple[Any, tuple] | None
        list
    i: int
        index of the element
        
    Returns
    -------
    Any
        element at the given index
        
    Raises
    ------
    ValueError
        exception if the index is out of bounds
    '''
    if is_empty(xs) or i < 0:
        raise ValueError('index out of bounds')
    if i == 0:
        return head(xs)
    return element_at(tail(xs), i - 1)

In [99]:
for i in range(length(l2)):
    print(element_at(l2, i))

a
b
c


Check whether the expected exception is raised when called with an index that is out of bounds.

In [114]:
try:
    _ = element_at(l1, 6)
except ValueError as e:
    print(f'Expected exception raised: {e}')

Expected exception raised: index out of bounds


In [115]:
try:
    _ = element_at(empty(), 0)
except ValueError as e:
    print(f'Expected exception raised: {e}')

Expected exception raised: index out of bounds


## Membership

We also need a function to check whether a value occurs in a list.

In [100]:
def contains(xs, x):
    '''Tests whether a value occurs in a list
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to search in
    x: Any
        value to search for
        
    Returns
    -------
    bool
        True if the value occurs in the list, False otherwise
    '''
    if is_empty(xs):
        return False
    if x == head(xs):
        return True
    return contains(tail(xs), x)

In [102]:
contains(l1, 2)

True

In [103]:
contains(l1, 5)

False

The empty set should not contain an element.

In [116]:
contains(empty(), 7)

False

## Inserting elements

A function to create a new list with a value inserted at a given index.

In [104]:
def insert_at(xs, x, i):
    '''Insert a value at a given position in the list
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to insert in
    x: Any
        value to be inserted
    i: int
        index to insert the value at
        
    Returns
    -------
    tuple[Any, tuple | None]
    
    Raises
    ------
    ValueError
        if the index is out of bounds
    '''
    if i == 0:
        return cons(x, xs)
    if is_empty(xs) or i < 0:
        raise ValueError('index out of bounds')
    return cons(head(xs), insert_at(tail(xs), x, i - 1))

In [105]:
insert_at(l1, 13, 2)

(1, (2, (13, (3, (4, None)))))

Note that the orignal list is not modified.

In [106]:
l1

(1, (2, (3, (4, None))))

Insert at the end of the list.

In [107]:
insert_at(l1, 13, 4)

(1, (2, (3, (4, (13, None)))))

Insert at the beginning of the list.

In [117]:
insert_at(l1, 13, 0)

(13, (1, (2, (3, (4, None)))))

Check that we can insert into an empty list at index 0.

In [109]:
insert_at(empty(), 13, 0)

(13, None)

Check that an exception is raised when the index is out of bounds.

In [108]:
try:
    insert_at(l1, 13, 5)
except ValueError as e:
    print(f'Expected exception: {str(e)}')

Expected exception: index out of bounds


In [118]:
try:
    insert_at(empty(), 13, 1)
except ValueError as e:
    print(f'Expected exception: {str(e)}')

Expected exception: index out of bounds


## Removing elements

We also want a function to create a new list with an element removed at the specified index.

In [119]:
def remove_at(xs, i):
    '''Remove an element from a list
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to remove an element from
    i: int
        index of the element to be removed
        
    Returns
    -------
    tuple[Any, tuple | None] | None
        list with the element removed
        
    Raises
    ------
    ValueError
        if the specified index is out of bounds
    '''
    if xs is None:
        raise ValueError('index out of bounds')
    if i == 0:
        return tail(xs)
    return cons(head(xs), remove_at(tail(xs), i - 1))

Remove the second element.

In [120]:
remove_at(l2, 1)

('a', ('c', None))

Remove the first element.

In [121]:
remove_at(l2, 0)

('b', ('c', None))

Remove the last element.

In [122]:
remove_at(l2, 2)

('a', ('b', None))

Removing an element from a list with a single element should return an empty list.

In [124]:
is_empty(remove_at(cons(5, empty()), 0))

True

Check whether an exception is thrown when the index is out of bounds.

In [125]:
try:
    remove_at(l2, 4)
except ValueError as e:
    print(f'Expected exception raised: {e}')

Expected exception raised: index out of bounds


In [126]:
try:
    remove_at(empty(), 0)
except ValueError as e:
    print(f'Expected exception raised: {e}')

Expected exception raised: index out of bounds


## Concatenate lists

A function that concatenates two lists is given below.

In [127]:
def concat(xs, ys):
    '''Create a list that is the concatenation of two lists
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        first list
    ys: tuple[Any, tuple | None] | None
        second list
        
    Returns
    -------
    tuple[Any, tuple | None] | None
        concatenation of the two given lists
    '''
    if is_empty(xs):
        return ys
    return cons(head(xs), concat(tail(xs), ys))

In [128]:
concat(l1, l2)

(1, (2, (3, (4, ('a', ('b', ('c', None)))))))

Check that this also works if one or both of the lists are empty.

In [129]:
concat(l1, empty())

(1, (2, (3, (4, None))))

In [130]:
concat(empty(), l2)

('a', ('b', ('c', None)))

In [132]:
is_empty(concat(empty(), empty()))

True

## Reverse list

A function that reverses a list is also easy to write.

In [136]:
def reverse(xs):
    '''Reverse the given list
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to reverse
        
    Returns
    -------
    tuple[Any, tuple | None] | None
        reversed list
    '''
    if is_empty(xs):
        return xs
    return concat(reverse(tail(xs)), cons(head(xs), empty()))

In [137]:
reverse(l1)

(4, (3, (2, (1, None))))

Check whether this works for an empty list.

In [138]:
is_empty(reverse(empty()))

True

## Minimum and maximum element

A function to search for the minimum or maximum element in a list can be

In [181]:
def extremum(xs, cmp):
    '''Find extremal element in the list based on the comparator function
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to search in
    cmp: function(Any, Any) -> bool
        function that returns True if its first parameter is less than its second parameter
        
    Returns
    -------
    Any
        extremal element in the list
        
    Raises
    ------
    ValueError
        if the list is empty
    '''
    if is_empty(xs):
        raise ValueError('empty list')
    if is_empty(tail(xs)):
        return head(xs)
    else:
        return cmp(head(xs), extremum(tail(xs), cmp))

Check whether we can find the minimum and maximum.

In [153]:
extremum(l1, cmp=min)

1

In [154]:
extremum(l1, cmp=max)

4

In [156]:
extremum(l3, cmp=min)

-1

In [157]:
extremum(l3, cmp=max)

7

Does it work for a list with a single element?

In [160]:
extremum(cons(13, empty()), cmp=min)

13

Check whether the expected exception is raised when calling this with an empty list.

In [159]:
try:
    extremum(empty(), cmp=min)
except ValueError as e:
    print(f'Expected exception raised: {e}')

Expected exception raised: empty list


## Filtering elements

We can write a simple function to filter list elements according to condition.

In [182]:
def select(xs, cond):
    '''Select elements that meet a given condition
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to select from
    cond: function(Any) -> bool
        function that returns True if a value meets a condition
        
    Returns
    -------
    tuple[Any, tuple | None] | None
        list with values that meet the given condition
    '''
    if is_empty(xs):
        return xs
    if cond(head(xs)):
        return cons(head(xs), select(tail(xs), cond=cond))
    else:
        return select(tail(xs), cond=cond)

In [162]:
select(l3, lambda x: x < 1)

(-1, (0, None))

In [163]:
select(l3, lambda x: x >= 1)

(3, (7, (2, None)))

## Sorting

We can trivially implement quicksort for this data structure.

In [179]:
def qsort(xs, cmp):
    '''Sort the elments of a list using the provided comparator
    
    Parameters
    ----------
    xs: tuple[Any, tuple | None] | None
        list to sort
    cmp: function(Any, Any) -> bool
        function that determines the order on the elements
        
    Returns
    -------
    tuple[Any, tuple | None] | None
        sorted list
    '''
    if is_empty(xs) or is_empty(tail(xs)):
        return xs
    return concat(qsort(select(tail(xs), lambda x: cmp(x, head(xs))), cmp=cmp),
                  concat(cons(head(xs), empty()),
                         qsort(select(tail(xs), lambda x: not cmp(x, head(xs))), cmp=cmp)
                        )
                 )

Pro memori, the list to sort is:

In [180]:
l3

(3, (-1, (7, (0, (2, None)))))

Sort in ascending order.

In [174]:
qsort(l3, lambda x, y: x < y)

(-1, (0, (2, (3, (7, None)))))

Sort in descending order.

In [175]:
qsort(l3, lambda x, y: y < x)

(7, (3, (2, (0, (-1, None)))))

Sort a sorted list.

In [176]:
qsort(l1, lambda x, y: x < y)

(1, (2, (3, (4, None))))

Sort a list that is sorted in reverse.

In [177]:
qsort(l1, lambda x, y: y < x)

(4, (3, (2, (1, None))))

Sort a list that only has a single element.

In [178]:
qsort(cons(13, empty()), lambda x, y: x < y)

(13, None)

Sort the empty list.

In [170]:
is_empty(qsort(empty()))

True